Ici on récupère les données via PISTE l'API permettant d'avoir accès aux données de Légifrance, le code de connexion OAuth est donné sur le Gitlab de Piste à cette [adresse](https://gitlab.com/piste_lab/oauth_connectors/-/blob/master/Python/Oauth2ClientCredentialsTest.py?ref_type=heads). Il est tout de même nécessaire d'installer les modules du code suivant.

In [22]:
%pip install python-dotenv
%pip install requests-oauthlib


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [23]:
import os
from dotenv import load_dotenv
import requests
from requests_oauthlib import OAuth2Session
import requests
import pandas as pd
import math
import json
from datetime import datetime, timedelta

In [38]:
def get_client():
    """
    Récupère un client OAuth2Session configuré avec un token d'accès depuis le serveur OAuth.

    :return: Un objet OAuth2Session prêt à être utilisé pour des requêtes API.
    """
    TOKEN_URL = "https://oauth.piste.gouv.fr/api/oauth/token"

    # Charger les identifiants client depuis le fichier .env
    load_dotenv()
    client_id = os.getenv("CLIENT_ID")
    client_secret = os.getenv("CLIENT_SECRET")

    # Requête pour obtenir le token
    res = requests.post(
        TOKEN_URL,
        data={
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
            "scope": "openid"
        }
    )

    res.raise_for_status()  # Lever une erreur si la requête échoue
    token = res.json()

    # Retourner un client OAuth2Session configuré
    return OAuth2Session(client_id, token=token)

In [25]:
def save_results_to_file(results, file_name, current_page):
    """
    Sauvegarde les résultats dans un fichier JSON. Si le fichier existe, ajoute les nouvelles données.

    :param results: Liste des résultats à sauvegarder.
    :param file_name: Nom du fichier JSON.
    :param current_page: La page actuelle traitée.
    """
    try:
        # Charger les données existantes si le fichier existe
        with open(file_name, "r", encoding="utf-8") as file:
            existing_data = json.load(file)
            if not isinstance(existing_data, dict):
                raise ValueError("Le fichier de sauvegarde n'est pas correctement structuré.")
            existing_results = existing_data.get("results", [])
            start_page = existing_data.get("current_page", 1)
    except (FileNotFoundError, ValueError):
        existing_results = []
        start_page = 1

    # Ajouter les nouveaux résultats
    existing_results.extend(results)

    # Sauvegarder les résultats mis à jour avec la page actuelle
    with open(file_name, "w", encoding="utf-8") as file:
        json.dump({"results": existing_results, "current_page": current_page}, file, ensure_ascii=False, indent=4)

In [26]:
def collect_all_results(api_host, code):
    # Nom unique pour le fichier de sauvegarde
    file_name = "results.json"

    # Charger la page de démarrage depuis le fichier de sauvegarde
    try:
        with open(file_name, "r", encoding="utf-8") as file:
            saved_data = json.load(file)
            if not isinstance(saved_data, dict):
                raise ValueError("Le fichier de sauvegarde n'est pas correctement structuré.")
            start_page = saved_data.get("current_page", 1)
    except (FileNotFoundError, ValueError):
        print(f"Fichier {file_name} introuvable ou mal structuré. Démarrage depuis la première page.")
        start_page = 1

    # Initialiser le client OAuth2Session et gérer le renouvellement
    client = get_client()
    expires_in = 55*60
    token_expiry = datetime.now() + timedelta(seconds=expires_in)  # Renouveler 1 minute avant expiration

    # Récupérer le total de résultats et calculer le nombre de pages
    response = client.post(api_host, json=code).json()
    total_results = response.get("totalResultNumber", 0)
    page_size = code["recherche"]["pageSize"]
    total_pages = math.ceil(total_results / page_size)

    print(f"Total de résultats : {total_results}")
    print(f"Nombre de pages à récupérer : {total_pages}")
    print(f"Reprise à partir de la page {start_page}")

    # Liste pour stocker les résultats courants
    all_results = []

    for page_number in range(start_page, total_pages + 1):
        # Vérifier si le token doit être renouvelé
        if datetime.now() >= token_expiry:
            print("Renouvellement du client OAuth...")
            client = get_client()
            token_expiry = datetime.now() + timedelta(seconds=expires_in)

        print(f"Récupération de la page {page_number}/{total_pages}...")
        code["recherche"]["pageNumber"] = page_number
        response = client.post(api_host, json=code).json()
        page_results = response.get("results", [])

        # Ajouter les résultats de la page courante
        all_results.extend(page_results)

        # Sauvegarder les résultats toutes les 20 pages ou à la dernière page
        if page_number % 20 == 0 or page_number == total_pages:
            print(f"Ajout des pages jusqu'à la page {page_number} dans {file_name}...")
            save_results_to_file(all_results, file_name, page_number)

            # Réinitialiser la liste des résultats sauvegardés
            all_results = []

    print(f"Récupération terminée. Dernière page sauvegardée : {total_pages}")
    return total_pages


In [27]:
API_HOST = "https://api.piste.gouv.fr/dila/legifrance/lf-engine-app"

code_api = {
    "recherche": {
        "champs": [
            {
                "typeChamp": "ALL",
                "criteres": [
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "délinquance", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "crime", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "délit", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Homicides", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Vols", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Stupéfiants", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Escroquerie", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Contrefaçon", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Sequestrations", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Recels", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Proxénétisme", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Menaces", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Cambriolages", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "infraction", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Attentats", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "dégradations", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Outrages", "operateur": "OU"}
                ],
                "operateur": "OU"
            }
        ],
        "pageNumber": 1,
        "pageSize": 100,
        "operateur": "ET",
        "sort": "DATE",
        "typePagination": "DEFAUT"
    },
    "fond": "ALL"
}

In [32]:
results = collect_all_results(API_HOST+"/search", code_api)

Fichier results.json introuvable ou mal structuré. Démarrage depuis la première page.
Total de résultats : 0
Nombre de pages à récupérer : 0
Reprise à partir de la page 1
Récupération terminée. Dernière page sauvegardée : 0


In [39]:
client = get_client()
r = client.post(API_HOST+"/search", code_api).json()
print(r)

{'timestamp': 1734298136076, 'error': 500, 'status': 'Internal Server Error', 'message': "Une exception non gérée est survenue, veuillez contacter l'administrateur", 'errorCode': None, 'exceptionDetails': None}


Pour pour envoyer des requêtes à l'API nous devons d'abord obtenir un token, après s'être inscrit sur le [site de PISTE](https://piste.gouv.fr/). Ce token est obtenu en envoyant une requête contenant mon identifiant client et mon code de client au site d'autorisation. Nous avons ensuite une autorisation d'une heure avec ce token pour exploiter l'API de Légifrance. 

Maintenant en utilisant le lien d'exploitation de l'API auquel on rajoute l'endpoints qui permet d'accéder à ce que l'on veut faire nous récupérons les données. La liste des endpoints pour l'API de Légifrance est disponible [ici](https://piste.gouv.fr/index.php?option=com_apiportal&view=apitester&usage=api&apitab=tests&apiName=L%C3%A9gifrance&apiId=7e5a0e1d-ffcc-40be-a405-a1a5c1afe950&managerId=3&type=rest&apiVersion=2.4.2&Itemid=179&swaggerVersion=2.0&lang=fr).

Le code ci-dessous permet de récupérer un ensemble de documents appartenant à l'ensemble des lois, ordonnance, décrets et arrêtés de Légifrance entre le premier janvier 1996 et le 31 août 2022 qui contiennent au moins un des mots d'une liste définie. Cette liste comprend les termes des taux de délinquances détaillés [ici](database_délinquance.ipynb#Calcul-des-taux-de-délinquance). Ce code est un json et sera envoyé par une méthode post à l'API, elle nous renverra un nombre de résultats limités (100 ici). Ce code est très inspiré de celui disponible sur le [site de l'API](https://piste.gouv.fr/index.php?option=com_apiportal&view=apitester&usage=api&apitab=tests&apiName=L%C3%A9gifrance&apiId=7e5a0e1d-ffcc-40be-a405-a1a5c1afe950&managerId=3&type=rest&apiVersion=2.4.2&Itemid=179&swaggerVersion=2.0&lang=fr) à l'endpoint /search.

On définit une fonction qui nous permet de récupérer tous les documents de notre recherche, car chaque recherche est limitée à une page de 100 éléments.

In [ ]:

def results_to_dataframe(results):
    data = []
    for result in results:
        title = result.get("titles", [{}])[0].get("title") if result.get("titles") else None
        date = result.get("date")
        id = result.get("titles", [{}])[0].get("id")
        data.append({"Titre": title, "Date": date, "ID": id})
    return pd.DataFrame(data)


In [ ]:
df_LODA = df_LODA.drop_duplicates()
df_LODA.to_csv("resultats_legifrance_loda.csv", index=False)

In [ ]:
df_CODE["date_version"] = pd.to_datetime(df["ID"].str.split("_").str[-1], format="%d-%m-%Y")
start_date = "1996-01-01"
end_date = "2022-08-31"
df_CODE = df_CODE[(df_CODE["date_version"] >= pd.to_datetime(start_date)) & (df_CODE["date_version"] <= pd.to_datetime(end_date))]
df_CODE.to_csv("resultats_legifrance_code.csv", index=False)